In [ ]:
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister
from qiskit_aer import aer_simulator
from qiskit import visualization



In [3]:
print("ENTER VALUES Range [a,b] :\n")

a= int(input())

b=int(input())

range = b-a+1
print(range)
qubit_numbers=0
while range>0:
    qubit_numbers+=1
    range//=2

print(qubit_numbers)


ENTER VALUES Range [a,b] :

30
5


In [ ]:
qubitstring = QuantumRegister(qubit_numbers,name= 'q')
Resultstring= ClassicalRegister(qubit_numbers)


# Draw circuit
qc = QuantumCircuit(qubitstring,Resultstring)

qc.h(qubitstring)

qc.measure(qubitstring, Resultstring)
qc.draw('mpl')


#execute
simulator = aer_simulator()
job= simulator.run(qc, shots=1024)
result = job.result()
count = result.getcounts()


bitstrings = list(count.keys())
firstvalid=None
for bitstring in bitstrings:
    val = int(bitstring, 2)
    if val <= b-a:  # if target in range
        firstvalid = val
        firstvalid+=a
        break

print(firstvalid)


TypeError: 'module' object is not callable